In [69]:
def classify_var_same_sub(subid='sub', trainList=['mem','mixed','motor'], predictList=['mem','mixed','motor']):
    import pandas as pd
    import numpy as np
    from mvpa2.datasets import dataset_wizard, vstack
    from mvpa2.generators.partition import NFoldPartitioner
    from mvpa2.measures.base import CrossValidation
    from mvpa2.clfs.meta import MappedClassifier
    from mvpa2.clfs.svm import LinearCSVMC
    #import mvpa2
    #import mvpa2.datasets
    import scipy.io
    #from mvpa2.suite import *
    #Consistent parameters to use for editing datasets
    nrois=333
    nsess=10
    nsub=10
    #make an empty df to store accuracy scores from each run
    accTable=[]
    for i in trainList:
        #Load task FC
        taskFile=scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/'+i+'/'+subid+'_parcel_corrmat.mat')
        #Convert to numpy array
        task_FC=np.array(taskFile['parcel_corrmat'])
        #Replace nans and infs with zero
        task_FC=np.nan_to_num(task_FC)
        #Index upper triangle of matrix
        task_mask=np.triu_indices(nrois,1)
        task_ds=np.empty((nsess, int(nrois*(nrois-1)/2)))
        task_value=0
        #Loop through all 10 days to reshape correlations into linear form
        for sess in range(nsess):
            tmp=task_FC[:,:,sess]
            task_ds[task_value]=tmp[task_mask]
            task_value=task_value+1
        #Load rest
        restFile=scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/rest/'+subid+'_parcel_corrmat.mat')
        rest_FC=np.array(restFile['parcel_corrmat'])
        rest_FC=np.nan_to_num(rest_FC)
        rest_mask=np.triu_indices(nrois,1)
        rest_ds=np.empty((nsess, int(nrois*(nrois-1)/2)))
        rest_value=0
        for sess in range(nsess):
            tmp=rest_FC[:,:,sess]
            rest_ds[rest_value]=tmp[rest_mask]
            rest_value=rest_value+1
        #Create a training dataset targets 1/0 ==task/rest, chunk=#days
        taskdf=dataset_wizard(samples=task_ds, targets=1, chunks=range(10))
        restdf=dataset_wizard(samples=rest_ds, targets=0, chunks=range(10))
        training=vstack((taskdf, restdf))
        #Linear SVM with cross validation scheme leave one out, output is the accuracy
        cv=CrossValidation(LinearCSVMC(), NFoldPartitioner(), 
                      enable_ca=['stats'],
                      errorfx=lambda p, t: np.mean(p == t))
        #Train the classifier         
        cv.train(training)
        acc_scores_per_task=[]
        #Lets loop through all scenerios for sub and train of interest
        for j in predictList:
            #Load testing set
            testFile=scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/'+j+'/'+subid+'_parcel_corrmat.mat')
            test_FC=np.array(testFile['parcel_corrmat'])
            test_FC=np.nan_to_num(test_FC)
            test_mask=np.triu_indices(nrois,1)
            test_ds=np.empty((nsess, int(nrois*(nrois-1)/2)))
            test_value=0
            for sess in range(nsess):
                tmp=test_FC[:,:,sess]
                test_ds[test_value]=tmp[task_mask]
                test_value=test_value+1
            #Create testing dataset
            testing=dataset_wizard(samples=test_ds, targets=1, chunks=range(10))
            #Time to train
            pre=cv(testing)
            #prediction accuracy
            acc=np.mean(pre)
            acc_scores_per_task.append(acc)
        tmp_df=pd.DataFrame({'Test_variables':predictList, i:acc_scores_per_task}).set_index('Test_variables')
        accTable.append(tmp_df)
    accTable=pd.concat(accTable, axis=1)
    accTable.to_csv('/Users/Alexis/Desktop/MSC_Alexis/analysis/output/results/MVPA/'+subid+'accTable_same_sub.csv')
        
    
    

In [61]:
classify_var_same_sub(subid='MSC01')

In [70]:
subs=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC08','MSC09','MSC10']
for sub in subs:
    classify_var_same_sub(sub)